In [3]:
import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from joblib import Parallel, delayed

from ccbo.core import (
    sim_espray_constrained,
    optimize_acqf_and_get_recommendation,
)

tkwargs = {
    "dtype": torch.double,
    # "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "device": torch.device("cpu"),
}

# Set the aiming particle size (um)
TARGET_SIZE = torch.tensor(6, **tkwargs)

# Set the solvent dictionary: 0 for DMAc, 1 for CHCl3
SOLVENT_DICT = {1: "CHCl3", 0: "DMAc"}

# Set the bounds for the processing parameters: Concentration (%), Flow Rates (uL/min), Voltage (kV), Solvent
bounds = torch.tensor([[0.05, 0.01, 10, 0], [5.0, 60.0, 18, 1]], **tkwargs)

# Set params for benchmark
ITERS = 10
TRIALS = 20
BATCH_SIZE = 2
strategies = ("qEI", "qEI_vi_mixed_con", "qEICF_vi_mixed_con", "rnd")

# Set if using parallel computing
PARA_EVAL = True

# exp_record organized by [Concentration (%), Flow Rate (uL/min), Voltage (kV), Solvent]
exp_record = torch.tensor(
    [
        [0.5, 15, 10, 0],
        [0.5, 0.1, 10, 1],
        [3, 20, 15, 0],
        [1, 20, 10, 1],
        [0.2, 0.02, 10, 1],
    ],
    **tkwargs
)

X_init = exp_record
Y_init = sim_espray_constrained(X_init)
print(Y_init)

tensor([[ 8.6440,  1.0000],
        [ 2.0731,  1.0000],
        [20.2264,  0.0000],
        [14.8625,  1.0000],
        [ 1.5904,  0.0000]], dtype=torch.float64)


## Constrained 

In [ ]:
def perform_trial(trial):
    # Set the random seed for reproducibility
    torch.manual_seed(trial)
    print("Trial:", trial)
    # Initialize the X and Y for each strategy
    X_bo = {k: X_init.clone() for k in strategies}
    Y_bo = {k: Y_init.clone() for k in strategies}

    # Initialize the result dictionary, best distance and raw_Y, for each strategy
    best_distance = {k: None for k in strategies}
    best_raw_Y = {k: None for k in strategies}

    for strategy in strategies:
        print(f"Performing Bayesian optimization using {strategy}...")
        best_distance_strategy = np.zeros(ITERS + 1)
        best_raw_Y_strategy = np.zeros(ITERS + 1)

        for iter in range(ITERS + 1):
            print("Iteration:", iter)
            # record the best distance and raw_Y at this iteration
            Y_distance = -np.abs(Y_bo[strategy][:, 0] - TARGET_SIZE)

            # get the best Y within the feasible region through masking out the infeasible points
            best_Y_ind = np.argmax(
                np.ma.masked_array(Y_distance, mask=~Y_bo[strategy][:, 1].bool())
            )
            raw_Y = Y_bo[strategy][best_Y_ind].numpy()

            best_distance_strategy[iter] = Y_distance.max().numpy()
            best_raw_Y_strategy[iter] = raw_Y[0]

            # perform the optimization and recommend next experiments
            new_x = optimize_acqf_and_get_recommendation(
                X_bo[strategy],
                Y_bo[strategy],
                bounds,
                y_target=TARGET_SIZE,
                strategy=strategy,
                batch_size=BATCH_SIZE,
            )

            X_bo[strategy] = torch.cat([X_bo[strategy], new_x])
            Y_bo[strategy] = torch.cat([Y_bo[strategy], sim_espray_constrained(new_x)])

        # log the best distances and raw_Y for this strategy over all iterations
        best_distance[strategy] = best_distance_strategy
        best_raw_Y[strategy] = best_raw_Y_strategy

    return best_distance, best_raw_Y, {"X": X_bo, "Y": Y_bo}
#perform the trials
if PARA_EVAL:
    results = Parallel(n_jobs=-1, verbose=80)(
        delayed(perform_trial)(trial) for trial in range(TRIALS)
    )
else:
    results = [perform_trial(trial) for trial in range(TRIALS)]


In [ ]:
#unpack the results
best_distances = {k: [] for k in strategies}
best_raw_Ys = {k: [] for k in strategies}
for result in results:
    best_distance, best_raw_Y, data = result
    for strategy in strategies:
        best_distances[strategy].append(best_distance[strategy])
        best_raw_Ys[strategy].append(best_raw_Y[strategy])

# vstack all values in best_distances
best_distances_vstack = {
    k: np.vstack(
        best_distances[k],
    )
    for k in strategies
}
best_distances_all_trials = -np.vstack([best_distances_vstack[k] for k in strategies])
best_distances_all_trials_df = pd.DataFrame(best_distances_all_trials)
best_distances_all_trials_df["strategy"] = np.repeat(
    ["Vanilla BO", "Constrained BO", "Constrained Composite BO", "rnd"], TRIALS
)
best_distances_all_trials_df["trial"] = list(range(TRIALS)) * len(strategies)

# conver into long format('qEI','qEI_vi_mixed_con','qEICF_vi_mixed_con','rnd')
best_distances_df_long = pd.melt(
    best_distances_all_trials_df,
    id_vars=["strategy", "trial"],
    var_name="iteration",
    value_name="regret",
)

# do the same for best_raw_Ys
best_raw_Ys_vstack = {
    k: np.vstack(
        best_raw_Ys[k],
    )
    for k in strategies
}
best_raw_Ys_all_trials = np.vstack([best_raw_Ys_vstack[k] for k in strategies])
best_raw_Ys_all_trials_df = pd.DataFrame(best_raw_Ys_all_trials)
best_raw_Ys_all_trials_df["strategy"] = np.repeat(
    ["Vanilla BO", "Constrained BO", "CCBO", "rnd"], TRIALS
)
best_raw_Ys_all_trials_df["trial"] = list(range(TRIALS)) * len(strategies)

# conver into long format
best_raw_Ys_df_long = pd.melt(
    best_raw_Ys_all_trials_df,
    id_vars=["strategy", "trial"],
    var_name="iteration",
    value_name="particle_size",
)

In [ ]:
# plot the results with error bars from multiple trials using seaborn
fs = 16
custom_params = {
    "font.family": "arial",
    "font.size": fs,
    "figure.dpi": 100,
    "lines.linewidth": 2,
    "lines.markersize": 8,
    "axes.labelsize": fs,
    "axes.linewidth": 0.7,
    "axes.titlesize": fs * 1.2,
    "xtick.labelsize": fs * 0.8,
    "ytick.labelsize": fs * 0.8,
    "xtick.major.width": 1,
    "xtick.major.size": 3,
    "ytick.major.width": 1,
    "ytick.major.size": 3,
    "legend.frameon": False,
    "legend.fontsize": fs * 0.6,
    "legend.markerscale": 0.7,
}
sns.set_theme(style="ticks", rc=custom_params)
# set palette
sns.set_palette("Set2")
# plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(4.5, 8))
sns.lineplot(
    data=best_distances_df_long[best_distances_df_long["strategy"] != "rnd"],
    x="iteration",
    y="regret",
    hue="strategy",
    ax=ax1,
    errorbar="se",
    alpha=0.7,
)
# plot strategy==rnd as gray
sns.lineplot(
    data=best_distances_df_long[best_distances_df_long["strategy"] == "rnd"],
    x="iteration",
    y="regret",
    ax=ax1,
    color="gray",
    errorbar="se",
    alpha=0.7,
)
# set labels
ax1.set_xlabel("Iteration")
ax1.set_ylabel("Regret (μm)")
ax1.legend("")

ax2.axhline(y=TARGET_SIZE, color="r", linestyle="--", label="Target Diameter")
sns.lineplot(
    data=best_raw_Ys_df_long[best_raw_Ys_df_long["strategy"] != "rnd"],
    x="iteration",
    y="particle_size",
    hue="strategy",
    ax=ax2,
    errorbar="se",
    alpha=0.7,
)
# plot strategy==rnd as gray
sns.lineplot(
    data=best_raw_Ys_df_long[best_raw_Ys_df_long["strategy"] == "rnd"],
    x="iteration",
    y="particle_size",
    ax=ax2,
    color="gray",
    errorbar="se",
    alpha=0.7,
    label="Random",
)
# Plot the target size (y_target) as a line

ax2.set_xlabel("Iteration")
ax2.set_ylabel("Particle Size (μm)")
ax2.legend(bbox_to_anchor=(0.45, 0.4), loc=2)
# reset

# Add title
plt.tight_layout()
plt.savefig(f"ccbo_benchmark_{TARGET_SIZE}um.png", dpi=300)